In [1]:
import sys
sys.path.append('C:\\Users\\Luan\\futebol\\Futebol')
import pandas as pd

In [2]:
from cafu.queries import Partida

In [3]:
req = Partida('365603')

In [4]:
campeonato = req.campeonato() # Retorna o campeonato.

In [5]:
campeonato

'Brasileirao 2013'

In [6]:
id_inicial = 598235

In [7]:
def dados_partida(id_):
    stop = False
    while not stop:
        try:
            req = Partida(str(id_))
            campeonato = req.campeonato()
            times = req.nomes_times()
            date = req.data()
            stop = True
        except:
            pass
    return campeonato, times, date

def id_left_right(id_inicial, campeonato_fix, left=True, partidas=None, dates=None, ids=None):
    if left:
        fator = -1
    else:
        fator = 1
    
    id_ = id_inicial 
    campeonato, times, date = dados_partida(id_)
    
    if partidas is None:
        partidas = [times]
    else:
        partidas.append(times)
    if dates is None:
        dates = [date]
    else:
        dates.append(date)
    if ids is None:
        ids = [id_]
    else:
        ids.append(id_)
        
    while campeonato == campeonato_fix:
        id_ = id_ + fator
        campeonato, times, date = dados_partida(id_)
        partidas.append(times)
        dates.append(date)
        ids.append(id_)
        if len(set(partidas)) < len(partidas):
            break
    
    id_ = id_ - fator
    partidas = partidas[:-1]
    dates = dates[:-1]
    ids = ids[:-1]
    
    return ids, partidas, dates

def ids_campeonato(id_inicial, campeonato_fix, qt_jogos_rodada=10):
    
    ids, partidas, dates = id_left_right(id_inicial, campeonato_fix)
    ids, partidas, dates = id_left_right(id_inicial+1, campeonato_fix, left=False, partidas=partidas, dates=dates, ids=ids)
    data = []
    for i in range(len(partidas)):
        data.append([ids[i], dates[i], partidas[i]])
    df = pd.DataFrame(data, columns=['jogo_id', 'dates', 'partida']).sort_values('jogo_id')
    df.index = range(df.shape[0])
    
    dates = []
    for i in range(0,df.shape[0], qt_jogos_rodada):
        median_date = df.iloc[i:i+qt_jogos_rodada].sort_values('dates')['dates'][i+int(qt_jogos_rodada/2)]
        dates_0 = [median_date for i in range(qt_jogos_rodada)]
        dates = dates+dates_0
    
    df['median_dates'] = dates
    
    df = df.sort_values(['median_dates'])
    rodadas = [i for i in range(1,(qt_jogos_rodada*2-1)*2+1) for j in range(qt_jogos_rodada)]
    df['rodada'] = rodadas
    df = df.drop('median_dates',axis=1)
    
    df = df.sort_values(['jogo_id'])
    
    df[['time_casa','time_visitante']] = pd.DataFrame(df['partida'].values.tolist(), index= df.index)
    df.drop('partida', axis=1)
    
    return df

In [32]:
id_inicial = 610557
campeonato = '2021-22 LaLiga'
df = ids_campeonato(id_inicial, campeonato)

610557 True 0 ('Granada', 'Osasuna') 2022-01-23 00:00:00
610556 True 1 ('Villareal', 'Mallorca') 2022-01-22 00:00:00
610555 True 2 ('Real Sociedad', 'Getafe') 2022-01-23 00:00:00
610554 True 3 ('Real Madrid', 'Elche') 2022-01-23 00:00:00
610553 True 4 ('Sevilla FC', 'Celta de Vigo') 2022-01-22 00:00:00
610552 True 5 ('Levante', 'Cádiz') 2022-01-22 00:00:00
610551 True 6 ('Alavés', 'Barcelona') 2022-01-23 00:00:00
610550 True 7 ('Rayo Vallecano', 'Athletic Bilbao') 2022-01-23 00:00:00
610549 True 8 ('Real Betis', 'Villareal') 2022-02-06 00:00:00
610548 True 9 ('Osasuna', 'Sevilla FC') 2022-02-05 00:00:00
610547 True 10 ('Valencia', 'Real Sociedad') 2022-02-06 00:00:00
610546 True 11 ('Celta de Vigo', 'Rayo Vallecano') 2022-02-05 00:00:00
610545 True 12 ('Getafe', 'Levante') 2022-02-04 00:00:00
610544 True 13 ('Real Madrid', 'Granada') 2022-02-06 00:00:00
610543 True 14 ('Athletic Bilbao', 'Espanyol') 2022-02-07 00:00:00
610542 True 15 ('Mallorca', 'Cádiz') 2022-02-05 00:00:00
610541 Tru

In [35]:
index_filter = df['rodada'] == 38
df.columns

Index(['jogo_id', 'dates', 'partida', 'rodada'], dtype='object')

In [19]:
df['partida'][0]

('Santos', 'Botafogo')

In [3]:
from cafu.utils import WebdriverChrome
path_driver = 'C:\\Users\\Luan\\futebol\\Futebol\\chromedriver.exe'
app = WebdriverChrome(path_driver)

In [4]:
app.web.get('https://www.espn.com.br/futebol/resultados/_/liga/eng.1')

In [8]:
ids = app.web.find_elements_by_xpath('//*[@id]')

In [10]:
ids[-1].get_attribute('id')

'112a8b4cbf734c1'

In [15]:
ids = app.web.find_elements_by_xpath('//*[@id]')

In [11]:
for ii in ids:
    #print ii.tag_name
    print (ii.get_attribute('id'))   # id name as string

twitter-wjs
font-link
disneyid-style
ad-slot-exclusions-665764
google_ads_iframe_/21783347309/espn.br/soccer/scoreboard_0__container__
ad-slot-overlay-435880
google_ads_iframe_/21783347309/espn.br/soccer/scoreboard_1__container__
google_ads_iframe_/21783347309/espn.br/soccer/scoreboard_1
fb-root
global-viewport
global-nav-mobile
header-wrapper
global-scoreboard
league-nav
conference-nav
fullbtn
leagues
global-header
global-nav-mobile-trigger
global-search-trigger
global-search
global-search-input
global-user-trigger
scores-link
global-scoreboard-trigger
global-nav
global-nav-secondary
pane-main
custom-nav
ad-slot-banner-967100
google_ads_iframe_/21783347309/espn.br/soccer/scoreboard_2__container__
google_ads_iframe_/21783347309/espn.br/soccer/scoreboard_2
main-container
scoreboard-page
sbpDate
events
605867
605863
605866
605865
vjs_video_3
vjs_video_3_html5_api
goog_883535852
vjs-playback-rate-value-label-vjs_video_3_component_327
autoplay_toggle
captions_toggle
audioWrapper
default
vj

In [19]:
ids[0].get_attribute('id')

'twitter-wjs'

In [ ]:
https://www.espn.com.br/futebol/resultados/_/liga/esp.1
    
    

In [15]:
id_campeonato = {'brasil1': 'bra.1',
                 'espanha1': 'esp.1'
                }

In [18]:
campeonato = 'espanha1'

In [19]:
app.web.get(f'https://www.espn.com.br/futebol/resultados/_/liga/{id_campeonato[campeonato]}')

In [29]:
web = app.web

In [31]:
xpath = '//*[@id="scoreboard-page"]/div[1]/div[3]/a'
web.find_element_by_xpath(xpath).click()

In [32]:
xpath = '//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[1]/table/thead/tr[1]/th[2]'
web.find_element_by_xpath(xpath).click()

In [33]:
xpath = '//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[2]/table/thead/tr/th[1]'
web.find_element_by_xpath(xpath).click()

In [ ]:
from datetime import date
def camp_to_date(date, web):
    """
    Dentro do site ESPN, na página de resultados do campeonato 
    https://www.espn.com.br/futebol/resultados/_/liga/<id_campeonato>, busca os resultados na data <date>
    
    Args:
        date: (str) data no formato 'dd-mm-yyyy'
    """
    year_atual = date.today().year
    year_date = int(date[-4:])
    month_date = int(date[3:5])
    
    xpath = '//*[@id="scoreboard-page"]/div[1]/div[3]/a'
    web.find_element_by_xpath(xpath)
    
    xpath = '//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[1]/table/thead/tr[1]/th[2]'
    web.find_element_by_xpath(xpath).click()
    
    #Fixando o ano
    for i in range(year_atual-year_date)
        xpath = '//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[2]/table/thead/tr/th[1]'
        web.find_element_by_xpath(xpath).click()
    
    #Fixando o mês
    xpath = f'//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[2]/table/tbody/tr/td/span[{month_date}]'
    web.find_element_by_xpath(xpath).click()
    
    #Fixando o dia
    xpath = '//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[1]/table/tbody/tr[3]/td[5]'
    web.find_element_by_xpath(xpath).click()
    day = web.find_element_by_xpath(xpath).text[13:].split(' ')[0]
    day = int(day)
    

In [ ]:
//*[@id="scoreboard-page"]/div[1]/div[2]/div/div/div[24]/a
//*[@id="scoreboard-page"]/div[1]/div[2]/div/div/div[22]/a/span


In [46]:
xpath = '//*[@id="sbpDate"]'
web.find_element_by_xpath(xpath).text[13:].split(' ')[0]

'5'

In [36]:
date = '01-07-1998'
date[3:5]

'07'

In [35]:
int('03')

3

In [37]:
days = web.find_elements_by_class_name('day')

In [39]:
for d in days:
    print(d.text)








































SEX
7
SAB
8
DOM
9
SEG
10
TER
11
QUA
12
QUI
13
SEX
14



















28
29
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
1
2
3
4
5
6
7
8


In [ ]:
//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[1]/table/tbody/tr[1]/td[4]
//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[1]/table/tbody/tr[1]/td[5]
//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[1]/table/tbody/tr[1]/td[6]
//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[1]/table/tbody/tr[1]/td[7]

//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[1]/table/tbody/tr[4]/td[5] #23
//*[@id="scoreboard-page"]/div[1]/div[3]/div/div[1]/table/tbody/tr[5]/td[6] #31